This hopefully demonstrates the difficulty I ran into. I want to make a generic N-grammer. Here I'm focusing on 2-grams just for simplicity. 

In [1]:
from pyspark.sql import SparkSession
from operator import add
import argparse
import numpy as np
import json
import string

def getTestPaths(args):
    if args==None: return (None,None)
    else :
        return (args[0],args[1])


def swap(tup):
    return (tup[1],tup[0])

if __name__ == "__main__":

    DATA_PATH  = "data/smaller.txt"
    N          = 2
    OUT_PATH   = "data/NGrams/smaller" + str(N)
    


In [2]:
    spark = SparkSession\
        .builder\
        .appName("Project0")\
        .getOrCreate()

    sc = spark.sparkContext


I try to make a list of RDDs that is as big as the size of the n-gram we're looking at. In this case it's just 2. Then I try to push each RDD into the list one at a time, with progressively one-off indices as their keys. In other words, the first rdd starts with its first key as 0, the second as -1, the third as -2, and so on. I want to join the RDDs later so that they line up correctly for the n-gram.

In [4]:
RDDs = []
for i in np.arange(N):
    print(i)
    X = sc.textFile(DATA_PATH)\
              .flatMap(lambda x: x.split(" "))\
              .zipWithIndex()\
              .mapValues(lambda x: x-i)\
              .map(lambda x: swap(x))
    print("from for, X: ",X.take(10))
    print("\n")
    RDDs.append(X)
    for i,rdd in enumerate(RDDs):
        print("from nested for, RDDs[%s]:"%i)
        print(RDDs[i].take(10))

0
from for, X:  [(0, 'Copyright'), (1, 'laws'), (2, 'are'), (3, 'changing'), (4, 'all'), (5, 'over'), (6, 'the'), (7, 'world.'), (8, 'Be'), (9, 'sure')]


from nested for, RDDs[0]:
[(0, 'Copyright'), (1, 'laws'), (2, 'are'), (3, 'changing'), (4, 'all'), (5, 'over'), (6, 'the'), (7, 'world.'), (8, 'Be'), (9, 'sure')]
1
from for, X:  [(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]


from nested for, RDDs[0]:
[(0, 'Copyright'), (1, 'laws'), (2, 'are'), (3, 'changing'), (4, 'all'), (5, 'over'), (6, 'the'), (7, 'world.'), (8, 'Be'), (9, 'sure')]
from nested for, RDDs[1]:
[(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]


As you can see from the print statements, everything seems to be working so far. But here is where shit gets weird. When I ry to take from the RDDs outside of that for loop I get unexpected results. I would think that RDD[0] has the first RDD which we created, indexing starting at one. Then the second one starts at -1, and so on. But for whatever reason, that's not what happens.

In [5]:
print("RDDs[0]: ",RDDs[0].take(10))
print("RDDs[1]: ",RDDs[1].take(10))

out of for, RDDs[0]:  [(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]
out of for, RDDs[1]:  [(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]


Above, it looks like RDDs[0] and RDDs[1] point to the same thing? But below, it's not so simle. 

In [6]:
for i,rdd in enumerate(RDDs):
        print("RDDs[%s]:"%i)
        print(RDDs[i].take(10))


RDDs[0]:
[(0, 'Copyright'), (1, 'laws'), (2, 'are'), (3, 'changing'), (4, 'all'), (5, 'over'), (6, 'the'), (7, 'world.'), (8, 'Be'), (9, 'sure')]
RDDs[1]:
[(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]


When we enumerate the list, it behaves appropriately. Why? The below code mimics the bad behavior though, even though we're using the enumerator again. What the hell is going on here? 

In [9]:
for i,rdd in enumerate(RDDs):
    print("i",i)
    print("RDDs[i]:", RDDs[i].take(10))  
    if i==0: 
        X = rdd
    else:
        print("before join, X: ",X.take(10))
        print()
        X = X.join(rdd)
        print("after join, X: ",X.take(10))
        print()
    print("new X: ",X.take(10))


i 0
RDDs[i]: [(0, 'Copyright'), (1, 'laws'), (2, 'are'), (3, 'changing'), (4, 'all'), (5, 'over'), (6, 'the'), (7, 'world.'), (8, 'Be'), (9, 'sure')]
new X:  [(0, 'Copyright'), (1, 'laws'), (2, 'are'), (3, 'changing'), (4, 'all'), (5, 'over'), (6, 'the'), (7, 'world.'), (8, 'Be'), (9, 'sure')]
i 1
RDDs[i]: [(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]
before join, X:  [(-1, 'Copyright'), (0, 'laws'), (1, 'are'), (2, 'changing'), (3, 'all'), (4, 'over'), (5, 'the'), (6, 'world.'), (7, 'Be'), (8, 'sure')]

after join, X:  [(0, ('laws', 'laws')), (4, ('over', 'over')), (8, ('sure', 'sure')), (12, ('copyright', 'copyright')), (16, ('country', 'country')), (20, ('redistributing', 'redistributing')), (24, ('other', 'other')), (28, ('', '')), (32, ('be', 'be')), (36, ('seen', 'seen'))]

new X:  [(0, ('laws', 'laws')), (4, ('over', 'over')), (8, ('sure', 'sure')), (12, ('copyright', 'copyright')), (16, 